In [1]:
# import the arcgis package
import arcgis

# set up ArcGIS Online credentials
gis = arcgis.GIS("home")


In [2]:
item_2020_census_housing = gis.content.get('81d9e89b8b574a649ff6e14f61c8494f')
lyr_counties = item_2020_census_housing.layers[2]

In [3]:
df_single_state = lyr_counties.query(
    where = "STATE = 'California'",
    as_df = True,
    return_geometry = True
)

In [4]:
field_aliases = {f['name']: f['alias'] 
              for f in lyr_counties.properties.fields}

df_single_state = df_single_state.rename(columns=field_aliases)

In [5]:
df_single_state = df_single_state.sort_values(
    'Total Housing Units',
    ascending=False
)

In [6]:
import seaborn as sns

In [7]:
sns.set_theme()

In [ ]:
sns.barplot(
    df_single_state,
    x = 'Name',
    y = 'Total Housing Units'

)

In [ ]:
# globally change the figure size
sns.set(rc={"figure.figsize": (9, 4)})

ax = sns.barplot(
    df_single_state,
    x = 'Name',
    y = 'Total Housing Units'

)

# rotate the labels on the X-Axis 90 degrees
ax = ax.set_xticklabels(df_single_state['Name'],rotation=90)

In [10]:
import plotly.express as px

In [ ]:
fig = px.bar(df_single_state, 
             x='Name', 
             y='Total Housing Units')
fig.show()

In [ ]:
df_single_state.dtypes

In [13]:
# keep only the necessary columns
df_for_fc = df_single_state[["Total Housing Units", "Name", 'SHAPE']]

# rename the column with no spaces in the name
df_for_fc = df_for_fc.rename(
    columns={
        "Total Housing Units": "total_housing_units"
    }
)

In [14]:
import arcpy
fc_memory = df_for_fc.spatial.to_featureclass("memory\\single_state")

In [ ]:
# create a chart object
arcpy_bar = arcpy.charts.Bar(
    x="Name",
    y = ["total_housing_units"],
    title = "Total Housing Units by County",
    dataSource = fc_memory,
    displaySize = (800, 400)
)

# sort the data by values
arcpy_bar.yAxis.sort='DESC'

# display the chart
arcpy_bar

In [16]:
import ipywidgets

drop_down_x = ipywidgets.Dropdown(
    options = df_single_state.columns,
    values = df_single_state.columns,
    description = "X Axis",
    value = 'Percent of housing units that are urban'
)

drop_down_y = ipywidgets.Dropdown(
    options = df_single_state.columns,
    values = df_single_state.columns,
    description = "Y Axis",
    value = 'Vacancy rate'
)


In [17]:
def show_a_chart(x_field, y_field):
    sns.scatterplot(df_single_state, x=x_field, y=y_field)

In [ ]:
ipywidgets.interact(show_a_chart,  # the function you wrote
                    x_field = drop_down_x,  # the x_field parameter in your function
                    y_field = drop_down_y # the y_field parameter in your function
                   )